Imported down all the necessary libraries which is thankfully just Pandas

In [1]:
import pandas as pd

Read the dataset using Pandas read_csv feature

In [2]:
dataset=pd.read_csv('bbb_odi_data.csv')

C:\Users\Vijay\AppData\Local\Temp\ipykernel_7008\2538919985.py:1: DtypeWarning: Columns (41,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv('bbb_odi_data.csv')


Filtered out the dataset to get the instances only where the Current run rate was <=5 during the course of the game

In [3]:
resultdf=dataset[(dataset['inns_rr']>6.50) & (dataset['inns_rr']<=8.00)]

There are several columns and I chose only the ones meaningful to the related query by using a list.

In [4]:
req_cols=['bat','batruns','ballfaced']

Now, my dataset is filtered out to get only the required columns

In [5]:
result=resultdf[req_cols]

Grouped sum of the required columns gives us the overall aggregate of the data. I've reset the columns as well to include the names as a row value in the dataset.

In [6]:
grouped=result.groupby(['bat']).sum().reset_index()

In [7]:
grouped

,bat,batruns,ballfaced
0,AB de Villiers,881,506
1,Aarif Sheikh,0,3
2,Aaron Finch,510,495
3,Aaron Hardie,3,5
4,Aaron Johnson,47,22
...,...,...,...
766,Zawar Farid,8,2
767,Zeeshan Maqsood,2,2
768,Ziaur Rahman,2,7
769,Zubin Surkari,16,24


I've used this different approach to sum the dismissals because there are some run out dismissals and not always it has been the striker who has gotten out, so based on the who_out? column values summation, I've found out the number of dismissals in this specific filter. 

In [8]:
dis=pd.DataFrame(resultdf['who_out?'].value_counts()).reset_index()

This is an attempt to make a similar column in both the dataframes, and use it for joins later.

In [9]:
dis.rename({'index':'bat'},axis=1,inplace=True)

In [10]:
dis

,bat,who_out?
0,Eoin Morgan,30
1,Virat Kohli,29
2,Rohit Sharma,24
3,David Warner,22
4,Moeen Ali,22
...,...,...
516,Sachith Pathirana,1
517,Nuwan Kulasekara,1
518,Akila Dananjaya,1
519,Amila Aponso,1


A left join has been performed on the left dataset which is the grouped data and then they've been joined on 'bat' column

In [11]:
finalresult=pd.merge(grouped,dis,how='left',on='bat')

Now, they've been sorted by descending order of 'batruns' and this operation has been performed in place to reflect the changes

In [12]:
finalresult.sort_values(by='batruns',ascending=False,inplace=True)

The resultant column names have been renamed for easier interpretation

In [13]:
finalresult.rename({'who_out?':'dismissals','batruns':'runs','ballfaced':'balls'},axis=1,inplace=True)

In [14]:
finalresult

,bat,runs,balls,dismissals
739,Virat Kohli,1986,1610,29.0
570,Rohit Sharma,1756,1209,24.0
299,Jason Roy,1477,1114,21.0
164,David Warner,1275,923,22.0
323,Jonny Bairstow,1243,886,16.0
...,...,...,...,...
529,Prasidh Krishna,0,3,1.0
215,Gary Ballance,0,2,NaN
553,Riaz Hassan,0,1,1.0
232,Greg Lamb,0,1,NaN


Before performing mathematical operations to find out the averages and strike rates, I've actually verified if the data types are suitable for calculations or not. Since both are of the same 'integer' type, it is easier for me to perform the operations.

In [15]:
finalresult.dtypes

bat            object
runs            int64
balls           int64
dismissals    float64
dtype: object

The averages and Runs Per Over (rpo) were calculated from the existing data and were rounded off to the 2 nearest decimal places.

In [16]:
finalresult['avg']=(finalresult['runs']/finalresult['dismissals'])
finalresult['avg']=round(finalresult['avg'],2)

In [17]:
finalresult['rpo']=(finalresult['runs']*6/finalresult['balls'])
finalresult['rpo']=round(finalresult['rpo'],2)

The end result has been saved to a csv file to use them further for visualizations on tableau

In [18]:
finalresult.to_csv('under8.csv')

In [19]:
finalresult

,bat,runs,balls,dismissals,avg,rpo
739,Virat Kohli,1986,1610,29.0,68.48,7.40
570,Rohit Sharma,1756,1209,24.0,73.17,8.71
299,Jason Roy,1477,1114,21.0,70.33,7.96
164,David Warner,1275,923,22.0,57.95,8.29
323,Jonny Bairstow,1243,886,16.0,77.69,8.42
...,...,...,...,...,...,...
529,Prasidh Krishna,0,3,1.0,0.00,0.00
215,Gary Ballance,0,2,NaN,NaN,0.00
553,Riaz Hassan,0,1,1.0,0.00,0.00
232,Greg Lamb,0,1,NaN,NaN,0.00
